![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

# Implicit Recommendation from ECommerce Data

Some of the material for this work is based on [A Gentle Introduction to Recommender Systems with Implicit Feedback](https://jessesw.com/Rec-System/) by Jesse Steinweg Woods. This tutorial includes an implementation of the Alternating Least Squares algorithm and some other useful functions (like the area under the curve calculation). Other parts of the tutorial are based on a previous version of the Implicit library and had to be reworked.

The dataset used for this work is from Kaggle [E-Commerce Data, Actual transactions from UK retailer](https://www.kaggle.com/carrie1/ecommerce-data)


# Global Imports

In [ ]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import implicit
import scipy
from sklearn import metrics
from pandas.api.types import CategoricalDtype

import wandb

In [ ]:
%run Common-Functions.ipynb

## Hyperparameter Tuning with Weights & Biases


In [ ]:
sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "prediction_auc", "goal": "maximize"},
    "parameters": {
        "percent_test": {"min":0.1, "max":0.3},
        "alpha": {"min":1, "max":30 },
        "factors" : {
            "values" : [64, 128]
        },
        "regularization": {"min":0.01, "max":.2},
        "iterations": {"min":20, "max":100}
    },
}

sweep_id = wandb.sweep(sweep_config, project="ecommerce")

def sweep():

    # Initialize a new wandb run
    with wandb.init() as run:

        selected_df = pd.read_pickle('../data/interim/ecommerce/selected_invoices.gz')   

        # The Sweep parameters are passed in with the wandb.config parameter
        invoices = list(np.sort(selected_df.InvoiceNo.unique())) # Get our unique customers
        products = list(selected_df.StockCode.unique()) # Get our unique products that were purchased
        quantity = list(selected_df.Quantity) # All of our purchases

        cols = selected_df.InvoiceNo.astype(CategoricalDtype(categories=invoices, ordered=True)).cat.codes 
        # Get the associated row indices
        rows = selected_df.StockCode.astype(CategoricalDtype(categories=products, ordered=True)).cat.codes 
        # Get the associated column indices
        purchases_sparse = scipy.sparse.csr_matrix((quantity, (rows, cols)), shape=(len(products), len(invoices)))
        product_train, product_test, products_altered, transactions_altered = make_train(purchases_sparse, pct_test = wandb.config['percent_test'])

        model = implicit.als.AlternatingLeastSquares(factors=wandb.config['factors'],
                                        regularization=wandb.config['regularization'],
                                        iterations=wandb.config['iterations'])

        alpha = wandb.config['alpha']                                                                  
        model.fit((product_train * alpha).astype('double'))

        user_vecs = model.user_factors
        item_vecs = model.item_factors

        test, popular = calc_mean_auc(product_train, products_altered, 
                      [scipy.sparse.csr_matrix(item_vecs), scipy.sparse.csr_matrix(user_vecs.T)], product_test)

        wandb.log({
            'prediction_auc': test
        })


wandb.agent(sweep_id, sweep, count=100)
